# Notebook 2: Entrenamiento de Modelos y Fine-Tuning
**Clasificación Multi-Label - Pascal 2007**

Este notebook realiza:
- Configuración de MLflow para tracking de experimentos
- Entrenamiento de 3 modelos pre-entrenados (ResNet50, EfficientNetB0, MobileNetV2)
- Comparación de resultados
- Fine-tuning del mejor modelo
- Guardado de modelos en MLflow

**Prerequisito:** Ejecutar [01_preparacion_datos.ipynb](01_preparacion_datos.ipynb) primero.

## 1. Importar Librerías

In [ ]:
import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss, accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import mlflow
import mlflow.keras

print(f"TensorFlow: {tf.__version__}")
print(f"MLflow: {mlflow.__version__}")

## 2. Cargar Configuración

In [ ]:
# Cargar configuración del notebook 1
config = np.load('../model_config.npy', allow_pickle=True).item()

IMG_SIZE = config['img_size']
BATCH_SIZE = config['batch_size']
EPOCHS = config['epochs']
TARGET_CLASSES = config['target_classes']
NUM_CLASSES = config['num_classes']

print("✅ Configuración cargada:")
print(f"   Tamaño imagen: {IMG_SIZE}x{IMG_SIZE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Epochs: {EPOCHS}")
print(f"   Clases: {TARGET_CLASSES}")
print(f"   Número de clases: {NUM_CLASSES}")

## 3. Cargar Datos Procesados

In [ ]:
# Cargar datos del notebook 1
data_dir = "processed_data"

print("Cargando datos procesados...")
X_train = np.load(os.path.join(data_dir, 'X_train.npy'))
y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
X_val = np.load(os.path.join(data_dir, 'X_val.npy'))
y_val = np.load(os.path.join(data_dir, 'y_val.npy'))
X_test = np.load(os.path.join(data_dir, 'X_test.npy'))
y_test = np.load(os.path.join(data_dir, 'y_test.npy'))

print(f"\n✅ Datos cargados:")
print(f"   Train: {X_train.shape}")
print(f"   Val:   {X_val.shape}")
print(f"   Test:  {X_test.shape}")

## 4. Configurar MLflow

In [ ]:
# Configurar MLflow para usar mlflow_data (carpeta padre)
import os
from pathlib import Path

# mlflow_data está en la carpeta padre del proyecto
project_root = Path.cwd().parent.parent  # Multi-Label_Classification_proyecto final
mlflow_data_dir = project_root / "mlflow_data"

print(f"📊 Usando MLflow en: {mlflow_data_dir}")
mlflow.set_tracking_uri(f"file://{str(mlflow_data_dir.absolute())}")

# Crear/obtener experimento
experiment_name = "MultiLabel_Pascal2007"
try:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"✅ Experimento creado: {experiment_name}")
except:
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if experiment:
        experiment_id = experiment.experiment_id
        print(f"✅ Experimento existente: {experiment_name}")
    else:
        # Si no existe, crear uno
        experiment_id = mlflow.create_experiment(experiment_name)
        print(f"✅ Experimento creado: {experiment_name}")

mlflow.set_experiment(experiment_name)
print(f"   ID: {experiment_id}")
print(f"\n💡 Para ver la UI de MLflow:")
print(f"   1. cd {mlflow_data_dir}")
print(f"   2. .\\mlflow_env\\Scripts\\Activate.ps1")
print(f"   3. mlflow ui --backend-store-uri ./")

## 5. Funciones de Utilidad

In [ ]:
def create_model(base_name):
    """Crea modelo con transfer learning."""
    if base_name == "ResNet50":
        base = keras.applications.ResNet50(
            weights='imagenet', 
            include_top=False, 
            input_shape=(IMG_SIZE, IMG_SIZE, 3)
        )
    elif base_name == "EfficientNetB0":
        base = keras.applications.EfficientNetB0(
            weights='imagenet', 
            include_top=False, 
            input_shape=(IMG_SIZE, IMG_SIZE, 3)
        )
    else:  # MobileNetV2
        base = keras.applications.MobileNetV2(
            weights='imagenet', 
            include_top=False, 
            input_shape=(IMG_SIZE, IMG_SIZE, 3)
        )

    # Congelar capas base
    base.trainable = False

    # Construir modelo
    model = keras.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='sigmoid')
    ], name=base_name)

    # Compilar
    model.compile(
        optimizer=keras.optimizers.Adam(0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model, base

def get_metrics(model, X, y):
    """Calcula métricas de evaluación."""
    predictions = (model.predict(X, verbose=0) >= 0.5).astype(int)
    
    return {
        'accuracy': accuracy_score(y.flatten(), predictions.flatten()),
        'precision': precision_score(y, predictions, average='macro', zero_division=0),
        'recall': recall_score(y, predictions, average='macro', zero_division=0),
        'f1_score': f1_score(y, predictions, average='macro', zero_division=0),
        'hamming_loss': hamming_loss(y, predictions)
    }

print("✅ Funciones definidas")

## 6. Función de Entrenamiento y Evaluación

In [ ]:
def train_and_evaluate(model_name):
    """Entrena y evalúa un modelo."""
    print(f"\n{'='*60}")
    print(f"   Entrenando: {model_name}")
    print(f"{'='*60}")

    # Crear modelo
    model, base = create_model(model_name)
    print(f"\nResumen del modelo:")
    model.summary()

    # Callbacks
    callbacks = [
        keras.callbacks.EarlyStopping(
            patience=3, 
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            factor=0.5, 
            patience=2,
            verbose=1
        )
    ]

    # Iniciar run de MLflow
    with mlflow.start_run(run_name=model_name):
        # Registrar parámetros
        mlflow.log_param("model", model_name)
        mlflow.log_param("img_size", IMG_SIZE)
        mlflow.log_param("batch_size", BATCH_SIZE)
        mlflow.log_param("epochs", EPOCHS)
        mlflow.log_param("classes", TARGET_CLASSES)
        mlflow.log_param("num_classes", NUM_CLASSES)
        mlflow.log_param("optimizer", "Adam")
        mlflow.log_param("learning_rate", 0.001)

        # Entrenar
        print(f"\nIniciando entrenamiento...")
        history = model.fit(
            x=X_train,
            y=y_train,
            validation_data=(X_val, y_val),
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            callbacks=callbacks,
            verbose=1
        )

        # Registrar métricas de entrenamiento
        for epoch in range(len(history.history['loss'])):
            mlflow.log_metric("train_loss", history.history['loss'][epoch], step=epoch)
            mlflow.log_metric("train_accuracy", history.history['accuracy'][epoch], step=epoch)
            mlflow.log_metric("val_loss", history.history['val_loss'][epoch], step=epoch)
            mlflow.log_metric("val_accuracy", history.history['val_accuracy'][epoch], step=epoch)

        # Calcular métricas de test
        print(f"\nEvaluando en conjunto de test...")
        metrics = get_metrics(model, X_test, y_test)

        # Registrar métricas finales
        for key, value in metrics.items():
            mlflow.log_metric(f"test_{key}", value)

        # Guardar modelo
        mlflow.keras.log_model(
            model, 
            artifact_path="model", 
            registered_model_name=model_name
        )

        # Registrar tags
        mlflow.set_tag("framework", "tensorflow")
        mlflow.set_tag("dataset", "Pascal2007")
        mlflow.set_tag("type", "transfer_learning")

        print(f"\n✅ Resultados de {model_name}:")
        for key, value in metrics.items():
            print(f"   {key}: {value:.4f}")

    # Limpiar memoria
    keras.backend.clear_session()
    del model, base
    gc.collect()

    return metrics, history

print("✅ Función de entrenamiento definida")

## 7. Entrenar Modelo ResNet50

In [ ]:
metrics_resnet, history_resnet = train_and_evaluate("ResNet50")

## 8. Entrenar Modelo EfficientNetB0

In [ ]:
metrics_efficient, history_efficient = train_and_evaluate("EfficientNetB0")

## 9. Entrenar Modelo MobileNetV2

In [ ]:
metrics_mobile, history_mobile = train_and_evaluate("MobileNetV2")

## 10. Comparación de Modelos

In [ ]:
# Crear DataFrame de comparación
results_df = pd.DataFrame({
    'ResNet50': metrics_resnet,
    'EfficientNetB0': metrics_efficient,
    'MobileNetV2': metrics_mobile
}).T

print("\n" + "="*70)
print("   COMPARACIÓN DE MODELOS")
print("="*70)
print(results_df.to_string())
print("="*70)

# Identificar mejor modelo
best_model_name = results_df['f1_score'].idxmax()
best_f1 = results_df.loc[best_model_name, 'f1_score']
print(f"\n🏆 MEJOR MODELO: {best_model_name} (F1-Score: {best_f1:.4f})")

# Guardar nombre del mejor modelo
with open('models/best_model_name.txt', 'w') as f:
    f.write(best_model_name)
print(f"✅ Mejor modelo guardado: {best_model_name}")

## 11. Visualizar Comparación

In [ ]:
# Gráfico de comparación
fig, axes = plt.subplots(1, 5, figsize=(18, 4))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for ax, col in zip(axes, results_df.columns):
    bars = ax.bar(results_df.index, results_df[col], color=colors)
    ax.set_title(col.replace('_', ' ').title(), fontsize=12, fontweight='bold')
    ax.set_ylim(0, 1)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(axis='y', alpha=0.3)
    
    # Agregar valores en las barras
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.suptitle('Comparación de Métricas por Modelo', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 12. Fine-Tuning del Mejor Modelo

In [ ]:
print(f"\n{'='*60}")
print(f"   Fine-Tuning de {best_model_name}")
print(f"{'='*60}")

# Crear modelo con todas las capas congeladas inicialmente
model_ft, base_ft = create_model(best_model_name)

# Callbacks para fine-tuning (más paciencia)
callbacks_ft = [
    keras.callbacks.EarlyStopping(
        patience=5, 
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        factor=0.5, 
        patience=3,
        verbose=1
    )
]

# Iniciar run de MLflow para fine-tuning
with mlflow.start_run(run_name=f"{best_model_name}_FineTuned"):
    # Registrar parámetros
    mlflow.log_param("model", f"{best_model_name}_FineTuned")
    mlflow.log_param("base_model", best_model_name)
    mlflow.log_param("img_size", IMG_SIZE)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("epochs", 20)
    mlflow.log_param("fine_tuning", True)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("learning_rate", 0.001)

    # Entrenar con más epochs
    print(f"\nEntrenando por 20 epochs adicionales...")
    history_ft = model_ft.fit(
        X_train, y_train, 
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE, 
        epochs=20, 
        callbacks=callbacks_ft, 
        verbose=1
    )

    # Registrar métricas de entrenamiento
    for epoch in range(len(history_ft.history['loss'])):
        mlflow.log_metric("train_loss", history_ft.history['loss'][epoch], step=epoch)
        mlflow.log_metric("train_accuracy", history_ft.history['accuracy'][epoch], step=epoch)
        mlflow.log_metric("val_loss", history_ft.history['val_loss'][epoch], step=epoch)
        mlflow.log_metric("val_accuracy", history_ft.history['val_accuracy'][epoch], step=epoch)

    # Calcular métricas finales
    print(f"\nEvaluando modelo fine-tuned...")
    metrics_ft = get_metrics(model_ft, X_test, y_test)

    # Registrar métricas finales
    for key, value in metrics_ft.items():
        mlflow.log_metric(f"test_{key}", value)

    # Guardar modelo
    mlflow.keras.log_model(
        model_ft, 
        artifact_path="model", 
        registered_model_name=f"{best_model_name}_FineTuned"
    )
    
    # También guardar en carpeta models para uso directo
    model_ft.save('models/best_model.keras')
    
    # Registrar tags
    mlflow.set_tag("framework", "tensorflow")
    mlflow.set_tag("dataset", "Pascal2007")
    mlflow.set_tag("fine_tuned", "True")
    mlflow.set_tag("type", "fine_tuning")

    print(f"\n✅ Resultados Fine-Tuning:")
    for key, value in metrics_ft.items():
        print(f"   {key}: {value:.4f}")

print(f"\n📊 Comparación:")
print(f"   Original F1:    {best_f1:.4f}")
print(f"   Fine-Tuned F1:  {metrics_ft['f1_score']:.4f}")
print(f"   Mejora:         {(metrics_ft['f1_score'] - best_f1):.4f} ({((metrics_ft['f1_score'] - best_f1) / best_f1 * 100):.2f}%)")

## 13. Visualizar Curvas de Aprendizaje

In [ ]:
# Gráfico de curvas de aprendizaje del fine-tuning
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history_ft.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history_ft.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_title('Pérdida durante Fine-Tuning', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy
axes[1].plot(history_ft.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[1].plot(history_ft.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[1].set_title('Exactitud durante Fine-Tuning', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 14. Resumen Final

In [ ]:
print("="*70)
print("   RESUMEN DE ENTRENAMIENTO")
print("="*70)
print(f"\n📊 Dataset: Pascal 2007")
print(f"🎯 Clases objetivo: {TARGET_CLASSES}")
print(f"🖼️  Tamaño de imagen: {IMG_SIZE}x{IMG_SIZE}")
print(f"\n🏗️  Modelos entrenados:")
print(f"   - ResNet50")
print(f"   - EfficientNetB0")
print(f"   - MobileNetV2")
print(f"\n🏆 Mejor modelo: {best_model_name}")
print(f"   F1-Score original: {best_f1:.4f}")
print(f"   F1-Score fine-tuned: {metrics_ft['f1_score']:.4f}")
print(f"\n💾 Modelos guardados en:")
print(f"   - MLflow: {os.path.abspath(mlflow_dir)}")
print(f"   - Local: models/best_model.keras")
print(f"\n✅ Entrenamiento completado exitosamente")
print("="*70)

---
## ✅ Notebook 2 Completado

**Siguiente paso:** Ejecutar [03_prediccion_reentrenamiento.ipynb](03_prediccion_reentrenamiento.ipynb) para realizar predicciones y reentrenar el modelo.